In [1]:
import torch
import numpy as np
from PIL import Image
from classify import HierarchicalClassifier

weights = "efficientnet_v2_s___hierarchical.state"
device, dtype = torch.device("cuda:0"), torch.bfloat16

classifier = HierarchicalClassifier(weights=weights, device=device, dtype=dtype)

In [2]:
import json, requests
from io import BytesIO

from typing import Union, List, Dict, Any

# Request structure:
# https://api.gbif.org/v1/species/1470/name

def get_field(obj : Dict[str, Any], field : Union[str, List[str]], key : Union[str, int]) -> Union[str, List[str]]:
    if isinstance(field, list):
        return [get_field(obj, f, key) for f in field]
    if field == "vernacularName":
        uri = f"https://api.gbif.org/v1/species/{key}/vernacularNames"
        response = requests.get(uri)
        js = json.load(BytesIO(response.content))
        possible_english_names = [res for res in js["results"] if res["language"] in ["", "eng", "en", "EN", "english", "ENGLISH", "us", "US"]]
        if possible_english_names:
            return possible_english_names[0]["vernacularName"]
        else:
            return "???"
    return obj[field]

def translateKey(key : Union[Union[int, str], List[Union[int, str]]], field : Union[str, List[str]]="scientificName") -> Union[str, List[str]]:
    if isinstance(key, list):
        return [translateKey(k) for k in key]
    uri = f"https://api.gbif.org/v1/species/{key}/name"
    response = requests.get(uri)
    js = json.load(BytesIO(response.content))

    return get_field(js, field, key)

In [5]:
idx_to_name = [{k : translateKey(v) for k, v in i2c.items()} for i2c in classifier.model.class_handles["idx_to_class"]]

In [7]:
class_to_name = [{i2c[k] : i2n[k] for k in i2c} for i2c, i2n in zip(classifier.model.class_handles["idx_to_class"], idx_to_name)]

In [14]:
# Clean scientific name; remove non-alphanumeric characters and replace spaces with underscores
import re
def clean_name(name : str) -> str:
    name_with_space = re.sub(r"[^a-zA-Z0-9 ]", "", name)
    parts = name_with_space.split(" ")
    genus = parts[0]
    if len(parts) == 1:
        species = ""
        author = ""
    elif parts[1].islower():
        species = parts[1]
        author = " ".join(parts[2:])
    else:
        species = ""
        author = " ".join(parts[1:])
    return f"{genus}-{species}-{author}".replace(" ", "_")

clean_class_to_name = [{k : clean_name(v) for k, v in c2n.items()} for c2n in class_to_name]

# Combine dicts to one
combined = dict()
for c2n in clean_class_to_name:
    combined.update(c2n)

with open("clean_class_to_name.json", "w") as f:
    json.dump(combined, f)